In [5]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from src.database import Base, Job, JobDetail, SessionLocal
from src.repository import JobRepository
import json

# Initialize repository
repo = JobRepository()

# Get all jobs without limit - fetch all records from the database
all_jobs_data = repo.get_all_jobs(limit=500, offset=0)

# Convert to DataFrame
df_main = pd.DataFrame(all_jobs_data)

print(f"Total jobs loaded: {len(df_main)}")

# Function to flatten lists for better display
def flatten_lists_for_display(df):
    """Create string representations of lists for better DataFrame display"""
    df_copy = df.copy()
    
    # Handle simple list columns (strings only)
    simple_list_columns = ['hard_skills', 'soft_skills', 'certifications', 'contact_emails', 'contact_phones']
    
    for col in simple_list_columns:
        if col in df_copy.columns:
            df_copy[f'{col}_str'] = df_copy[col].apply(
                lambda x: ', '.join(x) if isinstance(x, list) and x else '' if x else ''
            )
    
    # Handle complex list columns (with dictionaries)
    complex_list_columns = ['benefits', 'work_environment', 'professional_development', 
                           'work_life_balance', 'physical_requirements', 'work_conditions', 
                           'special_requirements']
    
    for col in complex_list_columns:
        if col in df_copy.columns:
            df_copy[f'{col}_str'] = df_copy[col].apply(
                lambda x: ', '.join(x) if isinstance(x, list) and x and isinstance(x[0], str) else 
                         ', '.join(x) if isinstance(x, list) and x else 
                         '' if x else ''
            )
    
    # Handle languages (list of dictionaries)
    if 'languages' in df_copy.columns:
        df_copy['languages_str'] = df_copy['languages'].apply(
            lambda x: ', '.join([f"{lang['language']} ({lang.get('proficiency', '')})" 
                                for lang in x if isinstance(x, list) and isinstance(lang, dict)]) 
            if isinstance(x, list) and x and len(x) > 0 and isinstance(x[0], dict) else
            ', '.join(x) if isinstance(x, list) and x else
            '' if x else ''
        )
    
    # For responsibilities, create a single string
    if 'responsibilities' in df_copy.columns:
        df_copy['responsibilities_str'] = df_copy['responsibilities'].apply(
            lambda x: '; '.join([r.get('description', '') for r in x if isinstance(x, list) and r]) if x and isinstance(x, list) else ''
        )
    
    return df_copy

# Apply the flattening
df_flat = flatten_lists_for_display(df_main)

# Show the processed job details with flattened lists
display_cols = ['job_title', 'company_name', 'city', 'country', 'min_salary', 
                'max_salary', 'seniority_level', 'employment_type', 'remote_work', 
                'hard_skills_str', 'soft_skills_str', 'responsibilities_str', 
                'languages_str', 'benefits_str', 'work_environment_str', 
                'professional_development_str', 'work_life_balance_str']

available_cols = [col for col in display_cols if col in df_flat.columns]

# Display the full table in Jupyter's GUI format
print("All processed job details:")
display(df_flat[available_cols])

Total jobs loaded: 31
All processed job details:


,job_title,company_name,city,country,min_salary,max_salary,seniority_level,employment_type,remote_work,hard_skills_str,soft_skills_str,responsibilities_str,languages_str,benefits_str,work_environment_str,professional_development_str,work_life_balance_str
0,Media buyer,adnero,Chișinău,Moldova,10000.0,50000.0,junior,full-time,on-site,"analysis, media planning, budget management, c...",,Launch Facebook ad campaigns with unlimited bu...,"Romanian (None), Russian (None)",Official employment with transparent payment s...,"Modern office, Flexible culture",,
1,Designer mobilă,ambianta,Chișinău,Moldova,NaN,NaN,junior,full-time,on-site,"AutoCad, SketchUp, 3D Max","Communication, Punctuality, Honesty, Correctne...",Perform tasks according to job description,"Romanian (None), Russian (None)","Negotiable salary, Opportunity for self-affirm...",Modern and comfortable workplace,Career progression and gaining good experience,
2,Barman,andy's,Chișinău,Moldova,NaN,NaN,junior,full-time,on-site,"Inventory management, Equipment operation, Dri...","Teamwork, Communication, Fast-paced, Ambitious",Prepare drinks according to tech sheets and co...,"Romanian (fluent), Russian (fluent)","Complete social package, Career advancement op...",,Training and continuous professional developme...,
3,Asamblor mobilă,ambianta,Sculeni,Moldova,NaN,NaN,junior,full-time,on-site,Furniture assembly,"responsible, punctual",Perform assembly tasks as per job description.,"Romanian (None), Russian (None)","Good working conditions, Attractive salary, Co...",,,
4,Consultant vânzări mobilă,accesit,Chișinău,Moldova,5000.0,20000.0,junior,full-time,on-site,"product presentation, client base expansion, s...","teamwork, communication, dynamism",present products; expand client base; support ...,"Romanian (None), Russian (None)","Official employment, Corporate communication, ...",Office in city center,,
5,Agent Comercial KA,acvilin grup srl,Chișinău,Moldova,17000.0,20000.0,mid,full-time,on-site,"client relationship management, sales, negotia...","result-oriented, people skills",Promote and sell company products; Collect and...,"Romanian (None), Russian (None)",Interesting work activity in a reputable compa...,,Training and support for personal and professi...,
6,Agent - consultant,alliance manpower s.r.l,Chișinău,Moldova,20000.0,NaN,junior,full-time,on-site,,"Responsibility, Activity, Communication, Punct...",Receive and consult clients at the office; Pro...,"Romanian (None), Russian (None)","Office work (full-time), Training and assistan...",office,Training and assistance from experienced manag...,
7,Vânzător,anvecomex,Cărpineni,Moldova,8000.0,15000.0,None,full-time,on-site,"Inventory management, Order processing, Custom...","Teamwork, Responsibility, Communication, Serio...",Consult clients and provide product/service in...,"Romanian (None), Russian (None)","Motivating salary package, Professional develo...","professional, stable",Professional development opportunities,
8,"Specialist în Marketing, Branding",amg jewelry,Chișinău,Moldova,NaN,NaN,mid,full-time,on-site,"Branding, Google Ads, Analytics, Digital marke...","Responsibility, Communication, Creativity, Per...",Develop and implement online and offline marke...,"Romanian (fluent), Russian (fluent), English (...","Professional and creative work environment, Op...",Professional and creative,Opportunities for personal and professional de...,
9,Croitor,arilux s.r.l.,Chișinău,Moldova,10000.0,NaN,junior,full-time,on-site,"Attention to detail, Leather cutting, Leather ...","Teamwork, Responsibility, Desire for perfection",Cut and trim leather materials according to pa...,"Romanian (None), Russian (None)","Professional growth opportunities, Friendly en...",Modern workplace,Professional growth opportunities,
